**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Input

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The act function gives the action to do for a given state based on the learned policy. The parameter `epsilon` determines, during training, when to deviate from the learned policy and when to explore new possible actions. It is thanks to this parameter that the algorithm is capable of learning. `epsilon` rules the trade-of between exploration and exploitation.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

# PARAMS

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=301 # set small when debugging
epochs_test=31 # set small when debugging

In [6]:
# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> The `position` array determines the current position of the mouse in the game.
>
> The `board` array determines the state of the game, where are the bonus and malus left.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    loss = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score += win - lose

        print("Win/lose count {}/{}. Average score ({})"
          .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.0/8.0. Average score (0.0)
Win/lose count 5.5/10.0. Average score (-2.25)
Win/lose count 10.0/17.0. Average score (-3.8333333333333335)
Win/lose count 10.0/14.0. Average score (-3.875)
Win/lose count 11.0/20.0. Average score (-4.9)
Win/lose count 12.0/25.0. Average score (-6.25)
Win/lose count 14.5/16.0. Average score (-5.571428571428571)
Win/lose count 9.5/14.0. Average score (-5.4375)
Win/lose count 10.5/15.0. Average score (-5.333333333333333)
Win/lose count 10.0/8.0. Average score (-4.6)
Win/lose count 14.0/15.0. Average score (-4.2727272727272725)
Win/lose count 7.0/16.0. Average score (-4.666666666666667)
Win/lose count 9.5/16.0. Average score (-4.8076923076923075)
Win/lose count 9.0/14.0. Average score (-4.821428571428571)
Win/lose count 10.5/18.0. Average score (-5.0)
Win/lose count 13.0/13.0. Average score (-4.6875)
Win/lose count 9.5/14.0. Average score (-4.676470588235294)
Win/lose count 9.0/6.0. Average score (-4.25)
Win/lose count 9.5/19.0. Average score (

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory[:-1] = self.memory[1:]
            self.memory[-1] = m

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [11]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        q = self.model.predict(s.reshape(-1, 5, 5, self.n_state)).reshape(-1, 1)
        return np.argmax(q)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        new_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        
        batch = [self.memory.random_access() for i in range(self.batch_size)]
               
        for i, (state, new_state, _, _, _) in enumerate(batch):
            input_states[i] = state
            new_states[i] = new_state
            
        q = self.model.predict(input_states)
        new_q = self.model.predict(new_states)
        target_q = q.copy()
            
        for i, (_, _, action, reward, game_over) in enumerate(batch):
            if game_over:
                target_q[i, action] = reward
            else:
                target_q[i, action] = reward + self.discount * np.max(new_q[i])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        width_model = 32
        
        model = Sequential([
            Flatten(input_shape=(5, 5, self.n_state)),
            Dense(width_model, activation='relu'),
            Dense(self.n_action),
        ])
                
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
from keras.layers import Flatten, Input


In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')

Epoch 000/301 | Loss 0.0199 | Win/lose count 5.5/8.0 (-2.5)
Epoch 001/301 | Loss 0.0077 | Win/lose count 1.5/3.0 (-1.5)
Epoch 002/301 | Loss 0.0205 | Win/lose count 6.0/11.0 (-5.0)
Epoch 003/301 | Loss 0.0067 | Win/lose count 5.5/6.0 (-0.5)
Epoch 004/301 | Loss 0.0129 | Win/lose count 3.0/3.0 (0.0)
Epoch 005/301 | Loss 0.0153 | Win/lose count 1.5/3.0 (-1.5)
Epoch 006/301 | Loss 0.0491 | Win/lose count 2.0/1.0 (1.0)
Epoch 007/301 | Loss 0.0101 | Win/lose count 2.5/2.0 (0.5)
Epoch 008/301 | Loss 0.0105 | Win/lose count 2.5/0 (2.5)
Epoch 009/301 | Loss 0.0100 | Win/lose count 2.5/4.0 (-1.5)
Epoch 010/301 | Loss 0.0051 | Win/lose count 2.0/3.0 (-1.0)
Epoch 011/301 | Loss 0.0649 | Win/lose count 5.0/3.0 (2.0)
Epoch 012/301 | Loss 0.0147 | Win/lose count 5.0/8.0 (-3.0)
Epoch 013/301 | Loss 0.1379 | Win/lose count 3.5/2.0 (1.5)
Epoch 014/301 | Loss 0.0031 | Win/lose count 2.5/1.0 (1.5)
Epoch 015/301 | Loss 0.0054 | Win/lose count 4.0/1.0 (3.0)
Epoch 016/301 | Loss 0.0086 | Win/lose count 5.5/

Epoch 138/301 | Loss 0.0021 | Win/lose count 5.5/1.0 (4.5)
Epoch 139/301 | Loss 0.0035 | Win/lose count 8.0/5.0 (3.0)
Epoch 140/301 | Loss 0.0020 | Win/lose count 15.0/1.0 (14.0)
Epoch 141/301 | Loss 0.0037 | Win/lose count 11.5/3.0 (8.5)
Epoch 142/301 | Loss 0.0036 | Win/lose count 4.5/3.0 (1.5)
Epoch 143/301 | Loss 0.0025 | Win/lose count 20.0/4.0 (16.0)
Epoch 144/301 | Loss 0.0023 | Win/lose count 23.0/2.0 (21.0)
Epoch 145/301 | Loss 0.0469 | Win/lose count 13.0/3.0 (10.0)
Epoch 146/301 | Loss 0.0093 | Win/lose count 6.5/0 (6.5)
Epoch 147/301 | Loss 0.0017 | Win/lose count 9.5/2.0 (7.5)
Epoch 148/301 | Loss 0.0051 | Win/lose count 12.5/4.0 (8.5)
Epoch 149/301 | Loss 0.0026 | Win/lose count 13.5/1.0 (12.5)
Epoch 150/301 | Loss 0.0038 | Win/lose count 20.0/0 (20.0)
Epoch 151/301 | Loss 0.0025 | Win/lose count 14.5/5.0 (9.5)
Epoch 152/301 | Loss 0.0027 | Win/lose count 7.5/3.0 (4.5)
Epoch 153/301 | Loss 0.0031 | Win/lose count 14.0/2.0 (12.0)
Epoch 154/301 | Loss 0.0016 | Win/lose coun

Epoch 275/301 | Loss 0.0069 | Win/lose count 14.0/0 (14.0)
Epoch 276/301 | Loss 0.0032 | Win/lose count 14.5/3.0 (11.5)
Epoch 277/301 | Loss 0.0032 | Win/lose count 5.0/3.0 (2.0)
Epoch 278/301 | Loss 0.0035 | Win/lose count 11.0/0 (11.0)
Epoch 279/301 | Loss 0.0040 | Win/lose count 15.0/2.0 (13.0)
Epoch 280/301 | Loss 0.0085 | Win/lose count 16.0/0 (16.0)
Epoch 281/301 | Loss 0.0037 | Win/lose count 9.5/1.0 (8.5)
Epoch 282/301 | Loss 0.0033 | Win/lose count 10.0/1.0 (9.0)
Epoch 283/301 | Loss 0.0032 | Win/lose count 15.5/2.0 (13.5)
Epoch 284/301 | Loss 0.0046 | Win/lose count 15.5/3.0 (12.5)
Epoch 285/301 | Loss 0.0536 | Win/lose count 15.5/3.0 (12.5)
Epoch 286/301 | Loss 0.0016 | Win/lose count 23.5/1.0 (22.5)
Epoch 287/301 | Loss 0.1050 | Win/lose count 8.5/0 (8.5)
Epoch 288/301 | Loss 0.0024 | Win/lose count 5.0/3.0 (2.0)
Epoch 289/301 | Loss 0.0026 | Win/lose count 7.0/1.0 (6.0)
Epoch 290/301 | Loss 0.0456 | Win/lose count 10.0/1.0 (9.0)
Epoch 291/301 | Loss 0.0052 | Win/lose count

In [14]:
HTML(display_videos('fc_train300.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        
        model = Sequential([
            Conv2D(32, (3, 3), input_shape=(5, 5, self.n_state)),
            Activation('relu'),
            Conv2D(16, (3, 3)),
            Activation('relu'),
            Flatten(),
            Dense(self.n_action),
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')

Epoch 000/301 | Loss 0.0034 | Win/lose count 6.0/9.0 (-3.0)
Epoch 001/301 | Loss 0.0114 | Win/lose count 3.0/3.0 (0.0)
Epoch 002/301 | Loss 0.0237 | Win/lose count 4.0/2.0 (2.0)
Epoch 003/301 | Loss 0.0025 | Win/lose count 7.0/3.0 (4.0)
Epoch 004/301 | Loss 0.1306 | Win/lose count 1.5/3.0 (-1.5)
Epoch 005/301 | Loss 0.0034 | Win/lose count 5.0/4.0 (1.0)
Epoch 006/301 | Loss 0.0055 | Win/lose count 1.0/2.0 (-1.0)
Epoch 007/301 | Loss 0.0414 | Win/lose count 2.0/3.0 (-1.0)
Epoch 008/301 | Loss 0.0046 | Win/lose count 8.0/3.0 (5.0)
Epoch 009/301 | Loss 0.0110 | Win/lose count 3.5/6.0 (-2.5)
Epoch 010/301 | Loss 0.0021 | Win/lose count 5.0/3.0 (2.0)
Epoch 011/301 | Loss 0.0047 | Win/lose count 5.0/8.0 (-3.0)
Epoch 012/301 | Loss 0.0056 | Win/lose count 4.0/3.0 (1.0)
Epoch 013/301 | Loss 0.0582 | Win/lose count 7.5/0 (7.5)
Epoch 014/301 | Loss 0.0062 | Win/lose count 11.0/1.0 (10.0)
Epoch 015/301 | Loss 0.0524 | Win/lose count 19.0/6.0 (13.0)
Epoch 016/301 | Loss 0.0054 | Win/lose count 15.

Epoch 137/301 | Loss 0.0020 | Win/lose count 11.0/1.0 (10.0)
Epoch 138/301 | Loss 0.0024 | Win/lose count 7.0/1.0 (6.0)
Epoch 139/301 | Loss 0.1043 | Win/lose count 17.5/5.0 (12.5)
Epoch 140/301 | Loss 0.0014 | Win/lose count 16.5/6.0 (10.5)
Epoch 141/301 | Loss 0.0024 | Win/lose count 18.0/3.0 (15.0)
Epoch 142/301 | Loss 0.0539 | Win/lose count 9.0/0 (9.0)
Epoch 143/301 | Loss 0.0032 | Win/lose count 7.5/3.0 (4.5)
Epoch 144/301 | Loss 0.0022 | Win/lose count 27.0/5.0 (22.0)
Epoch 145/301 | Loss 0.0022 | Win/lose count 11.5/3.0 (8.5)
Epoch 146/301 | Loss 0.0511 | Win/lose count 4.5/3.0 (1.5)
Epoch 147/301 | Loss 0.0024 | Win/lose count 7.0/0 (7.0)
Epoch 148/301 | Loss 0.0026 | Win/lose count 16.5/2.0 (14.5)
Epoch 149/301 | Loss 0.0016 | Win/lose count 18.0/3.0 (15.0)
Epoch 150/301 | Loss 0.0012 | Win/lose count 8.5/3.0 (5.5)
Epoch 151/301 | Loss 0.0106 | Win/lose count 19.0/2.0 (17.0)
Epoch 152/301 | Loss 0.0025 | Win/lose count 10.5/0 (10.5)
Epoch 153/301 | Loss 0.0567 | Win/lose coun

Epoch 273/301 | Loss 0.0026 | Win/lose count 20.5/5.0 (15.5)
Epoch 274/301 | Loss 0.0028 | Win/lose count 10.0/1.0 (9.0)
Epoch 275/301 | Loss 0.0033 | Win/lose count 10.5/3.0 (7.5)
Epoch 276/301 | Loss 0.0034 | Win/lose count 14.5/1.0 (13.5)
Epoch 277/301 | Loss 0.0021 | Win/lose count 17.0/3.0 (14.0)
Epoch 278/301 | Loss 0.0043 | Win/lose count 17.5/2.0 (15.5)
Epoch 279/301 | Loss 0.0031 | Win/lose count 20.0/3.0 (17.0)
Epoch 280/301 | Loss 0.0021 | Win/lose count 15.0/2.0 (13.0)
Epoch 281/301 | Loss 0.0019 | Win/lose count 17.5/4.0 (13.5)
Epoch 282/301 | Loss 0.0020 | Win/lose count 18.0/4.0 (14.0)
Epoch 283/301 | Loss 0.0028 | Win/lose count 25.0/3.0 (22.0)
Epoch 284/301 | Loss 0.0021 | Win/lose count 14.5/6.0 (8.5)
Epoch 285/301 | Loss 0.0049 | Win/lose count 9.5/1.0 (8.5)
Epoch 286/301 | Loss 0.0044 | Win/lose count 13.5/5.0 (8.5)
Epoch 287/301 | Loss 0.0030 | Win/lose count 12.5/4.0 (8.5)
Epoch 288/301 | Loss 0.0022 | Win/lose count 15.5/1.0 (14.5)
Epoch 289/301 | Loss 0.0128 | W

In [17]:
HTML(display_videos('cnn_train290.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [18]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 17.0/0. Average score (17.0)
Win/lose count 6.5/3.0. Average score (10.25)
Win/lose count 18.5/3.0. Average score (12.0)
Win/lose count 20.5/3.0. Average score (13.375)
Win/lose count 14.5/5.0. Average score (12.6)
Win/lose count 11.5/2.0. Average score (12.083333333333334)
Win/lose count 14.0/2.0. Average score (12.071428571428571)
Win/lose count 5.0/1.0. Average score (11.0625)
Win/lose count 16.0/2.0. Average score (11.38888888888889)
Win/lose count 9.5/3.0. Average score (10.9)
Win/lose count 12.0/5.0. Average score (10.545454545454545)
Win/lose count 12.5/3.0. Average score (10.458333333333334)
Win/lose count 16.5/4.0. Average score (10.615384615384615)
Win/lose count 14.0/3.0. Average score (10.642857142857142)
Win/lose count 16.5/2.0. Average score (10.9)
Win/lose count 17.5/2.0. Average score (11.1875)
Win/lose count 12.0/4.0. Average score (11.0)
Win/lose count 17.0/0. Average score (11.333333333333334)
Win/lose count 14.5/3.0. Average score (11.

In [19]:
HTML(display_videos('cnn_test30.mp4'))

In [20]:
HTML(display_videos('fc_test30.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [21]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [22]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, explore=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        self.explore = explore
        
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.malus_position[self.x, self.y] += self.explore
            
        self.t = self.t + 1
        reward = self.board[self.x, self.y] - self.malus_position[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [23]:
# ## In Environment exploring:
# # You will have to change n_state to 3 because you will use one more layer!
# reward = 0
# if train:
#     reward = -self.malus_position[self.x, self.y]
# self.malus_position[self.x, self.y] = 0.1

# reward = reward + self.board[self.x, self.y]
# # 3 "feature" states instead of 2
# state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                 self.board.reshape(self.grid_size, self.grid_size,1),
#                         self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [24]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3, explore=0.01)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore300.mp4'))

Epoch 000/301 | Loss 0.0159 | Win/lose count 6.370000000000002/19.09999999999999 (-12.72999999999999)
Epoch 001/301 | Loss 0.0104 | Win/lose count 3.920000000000001/28.20000000000002 (-24.28000000000002)
Epoch 002/301 | Loss 0.0040 | Win/lose count 5.880000000000002/25.510000000000005 (-19.630000000000003)
Epoch 003/301 | Loss 0.0087 | Win/lose count 2.9400000000000004/32.480000000000004 (-29.540000000000003)
Epoch 004/301 | Loss 0.0038 | Win/lose count 8.330000000000002/17.36999999999998 (-9.039999999999978)
Epoch 005/301 | Loss 0.0040 | Win/lose count 5.880000000000002/15.329999999999988 (-9.449999999999985)
Epoch 006/301 | Loss 0.0041 | Win/lose count 4.900000000000001/26.45 (-21.549999999999997)
Epoch 007/301 | Loss 0.0040 | Win/lose count 14.210000000000004/13.339999999999986 (0.8700000000000188)
Epoch 008/301 | Loss 0.0177 | Win/lose count 11.760000000000003/16.06999999999999 (-4.309999999999986)
Epoch 009/301 | Loss 0.0102 | Win/lose count 12.250000000000004/17.44 (-5.1899999999

Epoch 082/301 | Loss 0.0276 | Win/lose count 18.619999999999997/3.9799999999999893 (14.640000000000008)
Epoch 083/301 | Loss 0.0128 | Win/lose count 29.399999999999963/3.7099999999999937 (25.68999999999997)
Epoch 084/301 | Loss 0.0076 | Win/lose count 16.660000000000004/4.649999999999998 (12.010000000000005)
Epoch 085/301 | Loss 0.0098 | Win/lose count 22.539999999999985/6.219999999999989 (16.319999999999997)
Epoch 086/301 | Loss 0.0205 | Win/lose count 23.029999999999983/6.359999999999983 (16.67)
Epoch 087/301 | Loss 0.0116 | Win/lose count 22.539999999999985/8.449999999999985 (14.09)
Epoch 088/301 | Loss 0.0055 | Win/lose count 23.029999999999983/4.779999999999987 (18.249999999999996)
Epoch 089/301 | Loss 0.0063 | Win/lose count 22.049999999999986/8.469999999999985 (13.580000000000002)
Epoch 090/301 | Loss 0.0102 | Win/lose count 18.13/7.899999999999985 (10.230000000000015)
Epoch 091/301 | Loss 0.0094 | Win/lose count 23.519999999999982/5.039999999999979 (18.480000000000004)
Epoch 09

Epoch 164/301 | Loss 0.0416 | Win/lose count 21.559999999999988/7.129999999999988 (14.43)
Epoch 165/301 | Loss 0.0057 | Win/lose count 24.00999999999998/5.869999999999996 (18.139999999999986)
Epoch 166/301 | Loss 0.0106 | Win/lose count 15.190000000000005/5.599999999999992 (9.590000000000014)
Epoch 167/301 | Loss 0.0076 | Win/lose count 16.660000000000004/5.839999999999994 (10.820000000000011)
Epoch 168/301 | Loss 0.0437 | Win/lose count 22.539999999999985/8.68999999999997 (13.850000000000016)
Epoch 169/301 | Loss 0.0058 | Win/lose count 25.969999999999974/6.959999999999987 (19.009999999999987)
Epoch 170/301 | Loss 0.0058 | Win/lose count 24.989999999999977/6.679999999999979 (18.31)
Epoch 171/301 | Loss 0.0083 | Win/lose count 19.109999999999996/3.539999999999996 (15.57)
Epoch 172/301 | Loss 0.0048 | Win/lose count 25.969999999999974/5.819999999999989 (20.149999999999984)
Epoch 173/301 | Loss 0.0069 | Win/lose count 29.399999999999963/4.499999999999993 (24.89999999999997)
Epoch 174/301

Epoch 245/301 | Loss 0.0114 | Win/lose count 20.57999999999999/7.86999999999998 (12.710000000000012)
Epoch 246/301 | Loss 0.0068 | Win/lose count 25.969999999999974/5.709999999999992 (20.259999999999984)
Epoch 247/301 | Loss 0.0644 | Win/lose count 24.00999999999998/5.7699999999999845 (18.239999999999995)
Epoch 248/301 | Loss 0.0170 | Win/lose count 26.459999999999972/5.939999999999987 (20.519999999999985)
Epoch 249/301 | Loss 0.0050 | Win/lose count 24.00999999999998/5.289999999999988 (18.71999999999999)
Epoch 250/301 | Loss 0.0030 | Win/lose count 21.559999999999988/4.929999999999993 (16.629999999999995)
Epoch 251/301 | Loss 0.0067 | Win/lose count 19.109999999999996/7.519999999999981 (11.590000000000014)
Epoch 252/301 | Loss 0.0068 | Win/lose count 19.109999999999996/10.909999999999963 (8.200000000000033)
Epoch 253/301 | Loss 0.0060 | Win/lose count 25.969999999999974/5.569999999999994 (20.39999999999998)
Epoch 254/301 | Loss 0.0047 | Win/lose count 23.029999999999983/4.539999999999

In [25]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore30.mp4'))

Win/lose count 19.599999999999994/10.739999999999965. Average score (8.86000000000003)
Win/lose count 25.969999999999974/7.599999999999974. Average score (13.615000000000016)
Win/lose count 22.539999999999985/4.259999999999991. Average score (15.170000000000009)
Win/lose count 17.150000000000002/9.389999999999983. Average score (13.317500000000011)
Win/lose count 25.479999999999976/6.639999999999982. Average score (14.422000000000008)
Win/lose count 24.00999999999998/6.349999999999995. Average score (14.961666666666671)
Win/lose count 22.049999999999986/6.929999999999984. Average score (14.984285714285718)
Win/lose count 19.599999999999994/3.0399999999999974. Average score (15.181250000000002)
Win/lose count 25.479999999999976/4.849999999999999. Average score (15.786666666666665)
Win/lose count 23.029999999999983/6.819999999999984. Average score (15.828999999999999)
Win/lose count 19.599999999999994/5.409999999999993. Average score (15.68)
Win/lose count 19.109999999999996/5.9799999999

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***